<a href="https://colab.research.google.com/github/Khalil-Ravikson/Classificador_gatos_cachorros/blob/main/gatos_cachorros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First, install the main dependencies
!pip install numpy==1.25.0 pandas==2.0.3 tensorflow==2.16.1 scikit-learn==1.5.0 matplotlib==3.7.1 --quiet

# Then, install scikeras by itself
!pip install scikeras==0.13.0 --quiet

# Restart the runtime
import os
os.kill(os.getpid(), 9)

  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
import scikeras
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Input,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras import utils as np_utils
from tensorflow.keras import backend as k
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
import tempfile
import urllib
import zipfile

temp_dir = tempfile.TemporaryDirectory()
print(temp_dir)

<TemporaryDirectory '/tmp/tmpfyep72r8'>


In [ ]:
with zipfile.ZipFile("dataset.zip", 'r') as zip:
    zip.extractall(temp_dir.name)

In [ ]:
classificador = Sequential()
classificador.add (Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add (BatchNormalization())
classificador.add (MaxPooling2D(pool_size = (2, 2)))
classificador.add (Conv2D(32, (3, 3), activation = 'relu'))
classificador.add (BatchNormalization())
classificador.add (MaxPooling2D(pool_size = (2, 2)))
classificador.add (Flatten())

classificador.add (Dense(units = 128, activation = 'relu'))
classificador.add (Dropout(0.2))
classificador.add (Dense(units = 128, activation = 'relu'))
classificador.add (Dropout(0.2))
classificador.add (Dense(units = 1, activation = 'sigmoid'))

classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
classificador.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 29, 29, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 29, 29, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 829,985 (3.17 MB)

 Trainable params: 829,857 (3.17 MB)

 Non-trainable params: 128 (512.00 B)

In [ ]:
gerador_treinamento = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,rotation_range=7,
                                                                        shear_range = 0.2,
                                                                        zoom_range = 0.2,
                                                                        horizontal_flip = True,
                                                                      height_shift_range = 0.7)

gerador_teste = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [ ]:
base_treinamento = gerador_treinamento.flow_from_directory(f'{temp_dir.name}/dataset/training_set',target_size=(64,64),batch_size=32,class_mode='binary')

Found 4000 images belonging to 2 classes.


In [ ]:
base_teste = gerador_teste.flow_from_directory(f'{temp_dir.name}/dataset/test_set',target_size=(64,64),batch_size=32,class_mode='binary')

Found 1000 images belonging to 2 classes.


In [ ]:
classificador.fit(base_treinamento,epochs=10,validation_data=base_teste)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 55s 410ms/step - accuracy: 0.5307 - loss: 0.9980 - val_accuracy: 0.5000 - val_loss: 0.9623
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 54s 429ms/step - accuracy: 0.5568 - loss: 0.7127 - val_accuracy: 0.5080 - val_loss: 0.7400
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 48s 387ms/step - accuracy: 0.5802 - loss: 0.6822 - val_accuracy: 0.5590 - val_loss: 0.6913
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 52s 415ms/step - accuracy: 0.5871 - loss: 0.6574 - val_accuracy: 0.5400 - val_loss: 0.7148
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 82s 412ms/step - accuracy: 0.6028 - loss: 0.6723 - val_accuracy: 0.5300 - val_loss: 0.6841
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 49s 394ms/step - accuracy: 0.5782 - loss: 0.6650 - val_accuracy: 0.4990 - val_loss: 1.5017
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 52s 419ms/step - accuracy: 0.5977 - loss: 0.6590 - val_accuracy: 0.5790 - val_loss: 0.7044
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 50s 397ms/step - accuracy: 0.6147 - loss: 0

# testando o VCG16


In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
# Carrega a VGG16 sem as camadas do topo
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(64, 64, 3)) # Adapte o tamanho da imagem conforme seu dataset
# Congela as camadas para não serem treinadas
conv_base.trainable = False

In [ ]:
classificador2 = Sequential()

classificador2.add(conv_base)
classificador2.add (Flatten())


classificador2.add (Dense(units = 256, activation = 'relu'))
classificador2.add (Dropout(0.2))
classificador2.add (Dense(units = 1, activation = 'sigmoid'))

classificador2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
classificador2.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 2, 2, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,239,489 (58.13 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
classificador2.fit(base_treinamento,epochs=15,validation_data=base_teste)

Epoch 1/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 315s 3s/step - accuracy: 0.7547 - loss: 0.4950 - val_accuracy: 0.7810 - val_loss: 0.4554
Epoch 2/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 336s 3s/step - accuracy: 0.7663 - loss: 0.4747 - val_accuracy: 0.7840 - val_loss: 0.4548
Epoch 3/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 313s 3s/step - accuracy: 0.7582 - loss: 0.4857 - val_accuracy: 0.7750 - val_loss: 0.4600
Epoch 4/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 312s 3s/step - accuracy: 0.7600 - loss: 0.4855 - val_accuracy: 0.7870 - val_loss: 0.4541
Epoch 5/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 313s 3s/step - accuracy: 0.7586 - loss: 0.4861 - val_accuracy: 0.7830 - val_loss: 0.4534
Epoch 6/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 313s 3s/step - accuracy: 0.7756 - loss: 0.4607 - val_accuracy: 0.7870 - val_loss: 0.4546
Epoch 7/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 305s 2s/step - accuracy: 0.7676 - loss: 0.4635 - val_accuracy: 0.7880 - val_loss: 0.4555
Epoch 8/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 304s 2s/step - accuracy: 0.7589 - loss: 0.4813 - val_accu

In [ ]:
from tensorflow.keras.preprocessing import image
imagem_teste = image.load_img(f'{temp_dir.name}/dataset/test_set/gato/cat.3505.jpg', target_size = (64, 64))

In [ ]:
imagem_teste = image.img_to_array(imagem_teste)

In [ ]:
imagem_teste = imagem_teste/255

In [ ]:
import numpy as np
imagem_teste = np.expand_dims(imagem_teste, axis = 0)


In [ ]:
previsao = classificador2.predict(imagem_teste)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


In [ ]:
previsao >= 0.5
print(previsao)

[[0.90795624]]


In [ ]:
base_treinamento.class_indices

{'cachorro': 0, 'gato': 1}

In [ ]:
if previsao >= 0.5:
  print('Gato')
else:
  print('Cachorro')

Gato


In [ ]:
classificador.save('Classificador1.keras')
classificador2.save('Classificador2.keras')

In [ ]:
!ls

Classificador1.h5     Classificador2.h5     dataset.zip
Classificador1.keras  Classificador2.keras  sample_data


# usando streamlit


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.9 MB/s eta 0:00:00


In [ ]:
!npm install -q localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 4s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [ ]:
import streamlit as st
from tensorflow.keras.models import load_model

In [ ]:
from pyngrok import ngrok

# Substitua "<SEU_TOKEN_AQUI>" pelo token que você copiou do site do ngrok
ngrok.set_auth_token("317JPP1ioySw2vrywyhZCiN3BW7_5CCKmYFNYGsz1eaZMHY99")

print("Token de autenticação configurado com sucesso! 🎉")

Token de autenticação configurado com sucesso! 🎉


In [ ]:
import streamlit as st
from pyngrok import ngrok
from threading import Thread

# Crie o túnel do ngrok para o servidor do Streamlit
public_url = ngrok.connect(8501)
print("URL pública ngrok:", public_url)

# Use uma thread para rodar o Streamlit em segundo plano
def run_streamlit():
    !streamlit run seu_script.py &>/dev/null

thread = Thread(target=run_streamlit)
thread.start()

URL pública ngrok: NgrokTunnel: "https://3c1ffb6b78b3.ngrok-free.app" -> "http://localhost:8501"


In [ ]:

%%writefile app.py
import streamlit as st
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Title of the app
st.title("Classificador de Gato ou Cachorro")
st.write("Faça o upload de uma imagem e eu vou classificá-la como gato ou cachorro!")

# --- Load the trained model ---
@st.cache_resource # This decorator caches the model to avoid reloading it on every interaction
def load_my_model():
    # Make sure you have saved your model in the same directory as this script.
    # Ex: classifier_corrected.save('my_classifier.h5')
    model = load_model('/content/Classificador2.keras')
    return model

classifier = load_my_model()

# --- Image Upload and Classification Logic ---

uploaded_file = st.file_uploader("Escolha uma imagem...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display the uploaded image
    st.image(uploaded_file, caption='Imagem Carregada.', use_column_width=True)
    st.write("")
    st.write("Classificando...")

    try:
        # Load and preprocess the image
        img = image.load_img(uploaded_file, target_size=(64, 64))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) # Add a dimension for the batch
        img_array /= 255.0 # Normalize the image

        # Make the prediction
        prediction = classifier.predict(img_array)

        # Interpret the result
        if prediction[0][0] > 0.5:
            st.success("A imagem é um Gato!")
        else:
            st.success("A imagem é um Cachorro!")

    except Exception as e:
        st.error(f"Ocorreu um erro: {e}")

Overwriting app.py


/bin/bash: line 1: run: command not found


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙your url is: https://rotten-pants-rule.loca.lt
/content/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:21587 (check your firewall settings)
    at Socket.<anonymous> (/content/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙

In [ ]:
!pip install pyngrok --quiet